In [1]:
"""
MNIST - MNIST-M Domain Adaptation
"""

import tensorflow as tf

print(tf.__version__)

import numpy as np
import pandas as pd

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPool2D, BatchNormalization, Dropout

import os
import shutil
import sys
import matplotlib.pyplot as plt
import h5py

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import csc_dataloader

2.5.0


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def normalization (normal_df):
    
    min_max_scaler = preprocessing.MinMaxScaler()
    feature_names = list(normal_df)
    
    min_max_scaler = min_max_scaler.fit(normal_df.values)
    X_raw_minmax = min_max_scaler.transform(normal_df.values)
    normal_df = pd.DataFrame(X_raw_minmax, columns=feature_names)

    return normal_df, min_max_scaler

In [4]:
#CONSTANTS
#MNIST_M_PATH = './Datasets/MNIST_M/mnistm.h5'

BATCH_SIZE = 64
CHANNELS = 3
EPOCH = 10

source_df, target_df, tag_dict = csc_dataloader.loader('W4633070102', 'W4662FM0400')
source_df = source_df.sort_index()
target_df = target_df.sort_index()

source_df = csc_dataloader.labeler(source_df, 
                                    tag_dict['source_training_from'], 
                                    tag_dict['source_training_to'],
                                    tag_dict['source_end'])


target_df = csc_dataloader.labeler(target_df, 
                                    tag_dict['target_training_from'], 
                                    tag_dict['target_training_to'],
                                    tag_dict['target_end'])

source_x = source_df.drop(columns=['label'])
target_x = target_df.drop(columns=['label'])

source_x, normalizer = normalization(source_x)
target_x = normalizer.transform(target_x)

#target_y = tf.one_hot(target_df['label'], depth=2)
#source_y = tf.one_hot(source_df['label'], depth=2)


source_train_x, source_test_x, source_train_y, source_test_y = train_test_split(source_x, source_df['label'], test_size=640, shuffle=False)
target_train_x, target_test_x, target_train_y, target_test_y = train_test_split(target_x, target_df['label'], test_size=640, shuffle=False)

source_train_y = tf.one_hot(source_train_y, depth=2)
source_test_y = tf.one_hot(source_test_y, depth=2)
target_train_y = tf.one_hot(target_train_y, depth=2)
target_test_y = tf.one_hot(target_test_y, depth=2)


source_dataset = tf.data.Dataset.from_tensor_slices((source_train_x, source_train_y)).batch(BATCH_SIZE * 2)                                
da_dataset = tf.data.Dataset.from_tensor_slices((source_train_x, source_train_y, target_train_x, target_train_y )).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((source_test_x, source_test_y)).batch(BATCH_SIZE * 2) #Test Dataset over Target Domain
test_dataset2 = tf.data.Dataset.from_tensor_slices((target_test_x, target_test_y)).batch(BATCH_SIZE * 2) #Test Dataset over Target (used for training)

2021-07-27 16:08:22,553 W4633070102_W4662FM0400_data_loader
2021-07-27 16:08:22,561 Note: NumExpr detected 36 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-07-27 16:08:22,562 NumExpr defaulting to 8 threads.
2021-07-27 16:08:29,597 source shape:(100873, 396)
2021-07-27 16:08:29,599 target shape:(43518, 396)


In [5]:
#Gradient Reversal Layer
@tf.custom_gradient
def gradient_reverse(x, lamda=1.0):
    y = tf.identity(x)
    
    def grad(dy):
        return lamda * -dy, None
    
    return y, grad


class GradientReversalLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
    
    def call(self, x, lamda=1.0):
        return gradient_reverse(x, lamda)


class DANN(Model):
    def __init__(self):
        super().__init__()
        
        self.feature_extractor_layer0 = Dense(400, activation='relu')
        self.feature_extractor_layer1 = Dense(100, activation='relu')
        self.feature_extractor_layer2 = Dense(20, activation='relu')
        
        #Label Predictor
        self.label_predictor_layer0 = Dense(20, activation='relu')
        self.label_predictor_layer1 = Dense(10, activation='relu')
        self.label_predictor_layer2 = Dense(2, activation=None)
        
        #Domain Predictor
        self.domain_predictor_layer0 = GradientReversalLayer()
        self.domain_predictor_layer1 = Dense(20, activation='relu')
        self.domain_predictor_layer2 = Dense(2, activation=None)
        
    def call(self, x, train=False, source_train=True, lamda=1.0):
        #Feature Extractor
        x = self.feature_extractor_layer0(x)
        x = self.feature_extractor_layer1(x, training=train)
        x = self.feature_extractor_layer2(x)

        #print('DANN / feature shape:', x.shape)
        #feature = tf.reshape(x, [-1, 64])
        feature = x
        
        
        #Label Predictor
        if source_train is True:
            feature_slice = feature
        else:
            feature_slice = tf.slice(feature, [0, 0], [feature.shape[0] // 2, -1])
            
        #feature_slice = feature
        
        lp_x = self.label_predictor_layer0(feature_slice)
        lp_x = self.label_predictor_layer1(lp_x)
        l_logits = self.label_predictor_layer2(lp_x)
        #print('DANN / l_logits:', l_logits)
        
        #Domain Predictor
        if source_train is True:
            return l_logits
        else:
            dp_x = self.domain_predictor_layer0(feature, lamda)    #GradientReversalLayer
            dp_x = self.domain_predictor_layer1(dp_x)
            d_logits = self.domain_predictor_layer2(dp_x)
            
            #print('DANN / d_logits:', d_logits)
            
            return l_logits, d_logits




def loss_func(input_logits, target_labels):
    #print('loss_func / input_logits', input_logits)
    #print('loss_func / target_labels', target_labels)
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=input_logits, labels=target_labels))


def get_loss(l_logits, labels, d_logits=None, domain=None):
    #print('get_loss / l_logits', l_logits)
    #print('get_loss / labels', labels)
    
    if d_logits is None:
        return loss_func(l_logits, labels)
    else:
        return loss_func(l_logits, labels) + loss_func(d_logits, domain)

In [6]:
model = DANN()
model_optimizer = tf.optimizers.SGD()

domain_labels = np.vstack([np.tile([1., 0.], [BATCH_SIZE, 1]),
                           np.tile([0., 1.], [BATCH_SIZE, 1])])
domain_labels = domain_labels.astype('float32')


epoch_accuracy = tf.keras.metrics.CategoricalAccuracy()
source_acc = []  # Source Domain Accuracy while Source-only Training
da_acc = []      # Source Domain Accuracy while DA-training
test_acc = []    # Testing Dataset (Target Domain) Accuracy 
test2_acc = []   # Target Domain (used for Training) Accuracy

In [7]:
@tf.function
def train_step_source(s_images, s_labels, lamda=1.0):
    images = s_images
    labels = s_labels
    
    with tf.GradientTape() as tape:
        output = model(images, train=True, source_train=True, lamda=lamda)
        #print('output', output)
        #print('labels', labels)
        model_loss = get_loss(output, labels)
        epoch_accuracy(output, labels)
        
    gradients_mdan = tape.gradient(model_loss, model.trainable_variables)
    model_optimizer.apply_gradients(zip(gradients_mdan, model.trainable_variables))


@tf.function
def train_step_da(s_images, s_labels, t_images=None, t_labels=None, lamda=1.0):
    images = tf.concat([s_images, t_images], 0)
    labels = s_labels
    
    #print('train_step_da / s_labels', s_labels)
    
    with tf.GradientTape() as tape:
        output = model(images, train=True, source_train=False, lamda=lamda)

        #print('train_step_da / output', output)
        #print('train_step_da / labels', labels)
        
        model_loss = get_loss(output[0], labels, output[1], domain_labels)
        epoch_accuracy(output[0], labels)
        
    gradients_mdan = tape.gradient(model_loss, model.trainable_variables)
    model_optimizer.apply_gradients(zip(gradients_mdan, model.trainable_variables))


@tf.function
def test_step(t_images, t_labels):
    images = t_images
    labels = t_labels
    
    output = model(images, train=False, source_train=True)
    epoch_accuracy(output, labels)


def train(train_mode, epochs=EPOCH):
    
    if train_mode == 'source':
        dataset = source_dataset
        train_func = train_step_source
        acc_list = source_acc
    elif train_mode == 'domain-adaptation':
        dataset = da_dataset
        train_func = train_step_da
        acc_list = da_acc
    else:
        raise ValueError("Unknown training Mode")
    
    for epoch in range(epochs):
        p = float(epoch) / epochs
        lamda = 2 / (1 + np.exp(-100 * p, dtype=np.float32)) - 1
        lamda = lamda.astype('float32')

        for batch in dataset:
            #print('train_func batch', batch.shape)
            train_func(*batch, lamda=lamda)
        
        print("Training: Epoch {} :\t Source Accuracy : {:.3%}".format(epoch, epoch_accuracy.result()), end='  |  ')
        acc_list.append(epoch_accuracy.result())
        test()
        epoch_accuracy.reset_states()


def test():
    epoch_accuracy.reset_states()
    
    #Testing Dataset (Target Domain)
    for batch in test_dataset:
        test_step(*batch)
        
    print("Testing Accuracy : {:.3%}".format(epoch_accuracy.result()), end='  |  ')
    test_acc.append(epoch_accuracy.result())
    epoch_accuracy.reset_states()
    
    #Target Domain (used for Training)
    for batch in test_dataset2:
        test_step(*batch)
    
    print("Target Domain Accuracy : {:.3%}".format(epoch_accuracy.result()))
    test2_acc.append(epoch_accuracy.result())
    epoch_accuracy.reset_states()

In [ ]:
#Training
#train('source', 5)

train('domain-adaptation', EPOCH)


#Plot Results
x_axis = [i for i in range(0, EPOCH)]

plt.plot(x_axis, da_acc, label="source accuracy")
plt.plot(x_axis, test_acc, label="testing accuracy")
plt.plot(x_axis, test2_acc, label="target accuracy")
plt.legend()

Training: Epoch 0 :	 Source Accuracy : 91.143%  |  Testing Accuracy : 100.000%  |  Target Domain Accuracy : 100.000%
Training: Epoch 1 :	 Source Accuracy : 86.542%  |  Testing Accuracy : 100.000%  |  Target Domain Accuracy : 100.000%
Training: Epoch 2 :	 Source Accuracy : 86.393%  |  Testing Accuracy : 100.000%  |  Target Domain Accuracy : 100.000%
Training: Epoch 3 :	 Source Accuracy : 86.393%  |  Testing Accuracy : 100.000%  |  Target Domain Accuracy : 100.000%
Training: Epoch 4 :	 Source Accuracy : 86.393%  |  Testing Accuracy : 100.000%  |  Target Domain Accuracy : 100.000%
Training: Epoch 5 :	 Source Accuracy : 86.393%  |  Testing Accuracy : 100.000%  |  Target Domain Accuracy : 100.000%
Training: Epoch 6 :	 Source Accuracy : 86.393%  |  Testing Accuracy : 100.000%  |  Target Domain Accuracy : 100.000%
Training: Epoch 7 :	 Source Accuracy : 86.393%  |  Testing Accuracy : 100.000%  |  Target Domain Accuracy : 100.000%
Training: Epoch 8 :	 Source Accuracy : 86.393%  |  Testing Accur